# Fineian truthmaker semantics

In [72]:
{-# LANGUAGE OverloadedLists, FlexibleContexts, DeriveTraversable, ParallelListComp, LambdaCase, GADTs, KindSignatures, StandaloneDeriving #-}

import Data.Bifunctor
import Data.Semilattice.Join
import Data.Semilattice.Lower
import qualified Data.Map as M
import qualified Data.Set.Monad as S
import qualified Data.IntSet as IS
import Control.Monad (replicateM)
import Control.Applicative (liftA2)

Line 1: Unused LANGUAGE pragma
Found:
{-# LANGUAGE OverloadedLists, FlexibleContexts, DeriveTraversable, ParallelListComp, LambdaCase, GADTs, KindSignatures, StandaloneDeriving #-}
Why not:
{-# LANGUAGE OverloadedLists, FlexibleContexts, GADTs #-}

: 

## State space

Here we just take advantage of the fact that `IntSet` has a `Join` instance; states are sets of integers.

In [2]:
type S = IS.IntSet

Join semilattices give rise to a partial ordering, accessible via the `LessThan` wrapper.

In [3]:
compare (LessThan ([1] :: S)) (LessThan [1,2])

LT

## Syntax

First we recursively define a simple propositional language `Expr`.

In [4]:
data L a = Atom a | Not (L a) | (L a) :&: (L a) | (L a) :|: (L a) deriving (Foldable, Traversable, Functor)

type Expr = L Char

In truthmaker semantics *extensions*, are functions from truth-values to sets of states. This different from the usual presentation, but note that `Bool -> a` is isomorphic to `(a,a)`.

In [5]:
type Ext = Bool -> S.Set S

`variables` just gives back the set of variables in an expression.

In [6]:
variables :: Expr -> S.Set Char
variables = foldr S.insert S.empty

Assignments are functions from variables (i.e., characters) to extensions.

In [7]:
type Assignment = Char -> Ext

Some test data:

In [8]:
test1 = Atom 'a' :&: Atom 'b'

variables test1

fromList "ab"

## Semantics

The basic evaluation algorithm for Fine's sentential truthmaker semantics. N.b., 
- We use the `Semigroup` instance for `Set` for set union (i.e., `<>`).
- *Pointwise fusion is accomplished by lifting join (`\/`) into the `Set` applicative.

In [9]:
eval :: Assignment -> Expr -> Bool -> S.Set S
eval g (Atom a) = g a
eval g (Not expr) = eval g expr . not
eval g (lconj :&: rconj) = \t ->
        if t
            then
                let verifiers = (\conj -> eval g conj t)
                in liftA2 (\/) (verifiers lconj) (verifiers rconj)
            else
                let falsifiers = (\conj -> eval g conj t)
                in falsifiers lconj <> falsifiers rconj
eval g (ldisj :|: rdisj) = \t ->
        if t
            then
                let verifiers = (\disj -> eval g disj t)
                in verifiers ldisj <> verifiers rdisj
            else 
                let falsifiers = (\disj -> eval g disj t)
                in liftA2 (\/) (falsifiers ldisj) (falsifiers rdisj)

In [10]:
testg :: Assignment
testg 'a' True = S.fromList [[1],[3]]
testg 'a' False = S.fromList [[2],[4]]
testg 'b' True = S.fromList [[5],[7]]
testg 'b' False = S.fromList [[6],[8]]

eval testg test1 True
eval testg test1 False

fromList [fromList [1,5],fromList [1,7],fromList [3,5],fromList [3,7]]

fromList [fromList [2],fromList [4],fromList [6],fromList [8]]

# Desentential truthmaker semantics (riffing off Bledin)

N.b. that `(a,a)` is isomorphic to `Bool -> a`, and the latter representation gives rise to simpler computations, but using `(a,a)` allows us to avoid writing a bunch of show instances by hand.

In [119]:
instance (Join a, Join b) => Join (a,b) where
    (a,c) \/ (b,d) = (a \/ b, c \/ d)

type E = IS.IntSet

type Pol a = (a,a)

type EPol = Pol E

onePos :: Pol E
onePos = ([1],mempty)

oneNeg :: Pol E
oneNeg =  (mempty,[1])

twoPos :: Pol E
twoPos = ([2],mempty)
    
twoNeg :: Pol E
twoNeg = (mempty,[2])

type M a = (S.Set a, S.Set a)

vers :: M a -> S.Set a
vers = fst

fals :: M a -> S.Set a
fals = snd

_not :: M a -> M a
_not (x,y) = (y,x) -- flippity

_and :: Join a => M a -> M a -> M a
p `_and` q = (liftA2 (\/) (vers p) (vers q), fals p <> fals q)
    
_or :: Join a => M a -> M a -> M a
p `_or` q = (vers p <> vers q, liftA2 (\/) (fals p) (fals q))

In [96]:
_one :: M (Pol E)
_one = bimap pure pure (onePos,oneNeg)
    
_two :: M (Pol E)
_two = bimap pure pure (twoPos,twoNeg)

In [97]:
_one `_and` _two

(fromList [(fromList [1,2],fromList [])],fromList [(fromList [],fromList [1]),(fromList [],fromList [2])])

In [98]:
_one `_or` _two

(fromList [(fromList [1],fromList []),(fromList [2],fromList [])],fromList [(fromList [],fromList [1,2])])

In [103]:
-- this doesn't seem to be giving the right results

_one `_and` (_not _two)

Line 3: Redundant bracket
Found:
_one `_and` (_not _two)
Why not:
_one `_and` _not _two

(fromList [(fromList [1],fromList [2])],fromList [(fromList [],fromList [1]),(fromList [2],fromList [])])

In [106]:
vers (_not _two)

vers (_one)

Line 3: Redundant bracket
Found:
(_one)
Why not:
_one

fromList [(fromList [],fromList [2])]

fromList [(fromList [1],fromList [])]

## Predicates

_isHere :: M (Pol E -> Pol S)
_isHere = bimap pure pure (id,id)

biApp :: M a -> M (a -> b) -> M b
biApp (xs,ys) (fs,gs) = (fs <*> xs,gs <*> ys)

_one `biApp` _isHere

(_not _one) `biApp` _isHere

## Quantifiers

Let's start with a fixed domain.

In [164]:
dom :: [Int]
dom = [1..10]

Existential quantifiers via generalized disjunction:

In [168]:
_something :: M (Pol E)
_something = (S.fromList [([n], mempty) | n <- dom]
    , pure $ foldr (\/) mempty $ (S.fromList [(mempty,[n]) | n <- dom]))

vers _something
fals _something

Line 3: Redundant bracket
Found:
foldr (\/) mempty $ (S.fromList [(mempty, [n]) | n <- dom])
Why not:
foldr (\/) mempty $ S.fromList [(mempty, [n]) | n <- dom]Line 3: Redundant $
Found:
$
Why not:

fromList [(fromList [1],fromList []),(fromList [2],fromList []),(fromList [3],fromList []),(fromList [4],fromList []),(fromList [5],fromList []),(fromList [6],fromList []),(fromList [7],fromList []),(fromList [8],fromList []),(fromList [9],fromList []),(fromList [10],fromList [])]

fromList [(fromList [],fromList [1,2,3,4,5,6,7,8,9,10])]

Universal quantifiers via generalized conjunction

In [185]:
_everything :: M (Pol E)
_everything = (pure $ foldr (\/) mempty (S.fromList [([n],mempty) | n <- dom])
    , S.fromList [(mempty,[n]) | n <- dom])

_theThings :: M (Pol E)
_theThings = (pure $ foldr (\/) mempty (S.fromList [([n],mempty) | n <- dom])
    , pure $ foldr (\/) mempty (S.fromList [(mempty,[n]) | n <- dom]))

In [186]:
vers _everything
fals _everything

fromList [(fromList [1,2,3,4,5,6,7,8,9,10],fromList [])]

fromList [(fromList [],fromList [1]),(fromList [],fromList [2]),(fromList [],fromList [3]),(fromList [],fromList [4]),(fromList [],fromList [5]),(fromList [],fromList [6]),(fromList [],fromList [7]),(fromList [],fromList [8]),(fromList [],fromList [9]),(fromList [],fromList [10])]

In [187]:
vers _theThings

fromList [(fromList [1,2,3,4,5,6,7,8,9,10],fromList [])]

In [188]:
fals _theThings

fromList [(fromList [],fromList [1,2,3,4,5,6,7,8,9,10])]

## A problem with the currrent implementation

We don't currently predict homogeneity effects for conjoined NPs.

- TODO theory of presupposition: either there is a falsemaker or there is a truthmaker